# U.Group Coding Challenge Answers
## J. Mark Daniels

### Given there are no DateTimes this notebook proceeds under the assumption this is one month's worth of data and additional months will come in a similar csv file.

##### Modifications of this structure may be made to update systems in real time.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('cars.db')
c = conn.cursor()

In [3]:
c.execute('''SELECT *
            FROM Sales
            ;''')

df_Sales = pd.DataFrame(c.fetchall())
df_Sales.columns = [x[0] for x in c.description]

df_Sales.head()

,VIN,CarYear,Color,VehBody,EngineType,Make,Miles,Odometer,Brand,VehType,LocationNum,CarType,EngineLiters,FuelType,Transmission,SaleLoc,PurchVal
0,1G1AL15F777226903,2007,Silver,2 Door,4 Cylinder,COBALT LT,117029.0,Accurate,CHEVROLET,Passenger,86,Compact,2.2L,Gasoline,Automatic,86,4900
1,5NPEU46C66H126152,2006,Silver,4 Door,4 Cylinder,SONATA GLS/LX,108890.0,Accurate,HYUNDAI,Passenger,189,Full-size,2.4L,Gasoline,Automatic,189,5700
2,3MEHM08Z48R665374,2008,White,4 Door,4 Cylinder,MILAN,138000.0,Accurate,MERCURY,Passenger,360,Full-size,2.3L,Gasoline,Automatic,360,2890
3,1HGCP2F81AA000524,2010,Red,4 Door,4 Cylinder,ACCORD,89643.0,Accurate,HONDA,Passenger,302,Mid-size,2.4L,Gasoline,Automatic,302,9000
4,2G2WC52C951325364,2005,Maroon,4 Door,8 Cylinder,GRAND PRIX,127806.0,Accurate,PONTIAC,Passenger,295,Full-size,5.3L,Gasoline,Automatic,295,4000


### Attempted to Create VIN ID Column in Sales DataFrame

In [4]:
# index = range(0,(len(df_Sales['VIN'])))
# for x in index:
#     for vin in df_Sales['VIN']:
#         if vin == df_Sales['VIN'] :
#             print(df_vin['VIN_ID'][x])
        
#         for vin in df_vin['VIN']:
#             if df_Sales['VIN'][x] == vin:
#                 print(df_vin['VIN_ID'][x])

### Check to see if PurchVal values are Integers

In [5]:
df_Sales['PurchVal'][0]

4900

In [6]:
type(df_Sales['PurchVal'][0])

int

### Create Store Table

In [7]:
SaleStores = set([x for x in df_Sales['SaleLoc']])

In [8]:
LocStores = set([x for x in df_Sales['LocationNum']])

In [9]:
len(LocStores) == len(SaleStores)

True

In [10]:
len(LocStores)

192

In [11]:
df_stores = pd.DataFrame(LocStores)
df_stores.sort_values(by=[0],inplace=True)
df_stores['Store_ID'] = range(1,(len(LocStores)+1))
df_stores = df_stores.rename(columns={0: "Store"}, errors="raise")
df_stores

,Store,Store_ID
0,1,1
3,5,2
7,9,3
17,33,4
18,34,5
...,...,...
24,549,188
23,551,189
25,552,190
26,553,191


### Export to SQLite

In [12]:
df_stores.to_sql('Stores_Table', conn, if_exists='replace', index=False)
pd.read_sql('select * from Stores_Table', conn)

,Store,Store_ID
0,1,1
1,5,2
2,9,3
3,33,4
4,34,5
...,...,...
187,549,188
188,551,189
189,552,190
190,553,191


### Create VIN Table

In [13]:
VIN = set([x for x in df_Sales['VIN']])
len(VIN)

11367

In [14]:
df_vin = pd.DataFrame(VIN)
df_vin['VIN_ID'] = range(1,(len(VIN)+1))
df_vin = df_vin.rename(columns={0: "VIN"}, errors="raise")
df_vin

,VIN,VIN_ID
0,KMHCN46C36U058155,1
1,3G5DA03L36S563041,2
2,1FAFP53U05A194196,3
3,1HGCM66515A050352,4
4,1FMYU93137KA25131,5
...,...,...
11362,1G8ZS57B99F103167,11363
11363,JN8AZ28R99T120842,11364
11364,1G4HP52K45U134015,11365
11365,1G1AL58F787139541,11366


### Export to SQLite

In [15]:
df_stores.to_sql('VIN_Table', conn, if_exists='replace', index=False)
pd.read_sql('select * from VIN_Table', conn)

,Store,Store_ID
0,1,1
1,5,2
2,9,3
3,33,4
4,34,5
...,...,...
187,549,188
188,551,189
189,552,190
190,553,191


#### Check to see if same cars are going in and out of inventory

In [16]:
len(set(VIN))

11367

### Create Fact Table for All Sales Transactions

In [17]:
df_Sales.columns

Index(['VIN', 'CarYear', 'Color', 'VehBody', 'EngineType', 'Make', 'Miles',
       'Odometer', 'Brand', 'VehType', 'LocationNum', 'CarType',
       'EngineLiters', 'FuelType', 'Transmission', 'SaleLoc', 'PurchVal'],
      dtype='object')

In [18]:
df_Fact_Sales = pd.DataFrame(df_Sales['PurchVal'])

df_Fact_Sales

,PurchVal
0,4900
1,5700
2,2890
3,9000
4,4000
...,...
22783,3800
22784,4800
22785,5740
22786,4400


### Export to SQLite

In [19]:
df_stores.to_sql('Fact_Table', conn, if_exists='replace', index=False)
pd.read_sql('select * from Fact_Table', conn)

,Store,Store_ID
0,1,1
1,5,2
2,9,3
3,33,4
4,34,5
...,...,...
187,549,188
188,551,189
189,552,190
190,553,191


### Create Aggregation Features